In [531]:
import subprocess, os, cv2, math
import pandas as pd

PROJECT_FOLDER = os.path.expanduser(f'~/Desktop/TwoManakin/')
SSD_PATH = os.path.abspath('/Volumes/Extreme SSD/PROJECT MANACUS')


def open_folder(path):
    subprocess.run(["open", path])

def index_to_path(sheet, indx):
    choice = sheet.iloc[indx]
    cols = sheet.columns[:5]
    base_folder = '/Volumes/Extreme SSD/PROJECT MANACUS'

    for next_col in cols:
        base_folder = os.path.join(str(base_folder), str(choice[next_col]))

    if os.path.exists(base_folder):
        return base_folder
    else:
        return False

def make_spreadsheet():
    spreadsheet_folder_path = os.path.join(PROJECT_FOLDER, 'manacus_spreadsheets')
    sheets = []
    for i in os.listdir(spreadsheet_folder_path):
        if '.xlsx' in i:
            sheet = pd.read_excel(os.path.join(spreadsheet_folder_path, i))
            if 'Lek' in i:
                sheet.columns = sheet.iloc[0]
                sheet.drop(0, axis=0, inplace=True)
            sheets.append(sheet)
    spreadsheet = pd.concat(sheets)
    spreadsheet = spreadsheet[['Folder', 'Lek', 'Pista', 'DateRange', 'FileName', 'Male', 'PresentYN', 'DisplayYN', 'Snaps', 'quietSnaps', 'Grunts', 'Rolls', 'Calls', 'FemVisitation', 'Copulation', 'JuvPresent', 'Gardening', 'Notes', 'Observer']]
    spreadsheet = spreadsheet.replace({'Y': 1, 'y': 1, 'N': 0, 2: 1})
    spreadsheet = spreadsheet[(spreadsheet['PresentYN'] != 'PresentYN') & (spreadsheet['FemVisitation'] != 'FemVisitation') & (spreadsheet['FemVisitation'] != 'Maybe')].reset_index(drop=True)
    return spreadsheet

def sample_files(spreadsheet, n=10):
    temp = spreadsheet
    temp['Place'] = temp['Lek'] + ' ' + temp['Pista']
    sample_df = pd.DataFrame()
    for combo in temp['Place'].unique():
        subset = temp[temp['Place'] == combo]
        if len(subset) >= n:
            subset = subset.sample(n=n)
        sample_df = pd.concat([sample_df, subset])
    return sample_df.reset_index(drop=True)


def euclidean_distance(dot1, dot2):
    x1, y1 = dot1
    x2, y2 = dot2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)


def highlight(ssdfilepath):
    if os.path.exists(ssdfilepath):
        splits = ssdfilepath.split('/')
        lek, pista, vidname = splits[5], splits[6], splits[8]
        arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/"
        if not os.path.exists(arena):
            os.makedirs(arena)

        all_centers = []
        
        open_folder(arena)
        cap = cv2.VideoCapture(ssdfilepath)
        ret, prev_frame_color = cap.read()  # Initialize prev_frame with the first frame
        if not ret:
            print("Could not read the first frame.")
        frame_count = 0
        
        while cap.isOpened():
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 20)
            frame_count += 1
            ret, frame_color = cap.read()
            if not ret:
                break
            diff_color = cv2.absdiff(prev_frame_color, frame_color)
            diff_gray = cv2.cvtColor(diff_color, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(diff_gray, 30, 255, cv2.THRESH_BINARY)

            contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]             
                for box in [cv2.boundingRect(cnt) for cnt in contours]:
                    x, y, w, h = box
                    center = (x + w // 2, y + h // 2)

                    crop_x1 = max(center[0] - width, 0)
                    crop_x2 = min(center[0] + width, frame_color.shape[1])
                    crop_y1 = max(center[1] - width, 0)
                    crop_y2 = min(center[1] + width, frame_color.shape[0])

                    cropped_image = cv2.resize(frame_color[crop_y1:crop_y2, crop_x1:crop_x2], (100, 100))

                    
                    keep_dot = True
                    for c in all_centers:
                        distance = euclidean_distance(c, center)
                        if distance < 5:
                            keep_dot = False
                            break
                    
                    if keep_dot == True:
                        known_class = 0
                        all_centers.append(center)
                        image_filename = os.path.join(arena, f"{center[0]}, {center[1]}, {known_class}.jpg")
                        cv2.imwrite(image_filename, cropped_image)
                        cv2.circle(frame_color, center, 3, (0, 0, 255), -1)
                    cv2.imshow(f"{ssdfilepath.split('/')[5] + ' ' + ssdfilepath.split('/')[6]}", frame_color)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            prev_frame_color = frame_color  # Update prev_frame for the next iteration
        cap.release()
        cv2.destroyAllWindows()
        print(len(all_centers))
        
    else:
        print(f"Could not open video because ssdfilepath '{ssdfilepath}' does not exist.")

In [155]:
lek = 'Lek 14'
pista = 'Pista 2 (YYB)'
arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/"
os.listdir(arena)

['429, 319, 0.jpg',
 '1608, 925, 0.jpg',
 '688, 1036, 0.jpg',
 '1808, 995, 0.jpg',
 '1311, 783, 0.jpg',
 '943, 985, 0.jpg',
 '431, 853, 0.jpg',
 '1044, 616, 0.jpg',
 '1472, 799, 0.jpg',
 '1837, 400, 0.jpg',
 '1042, 525, 0.jpg',
 '192, 335, 0.jpg',
 '319, 687, 0.jpg',
 '1896, 507, 0.jpg',
 '324, 512, 0.jpg',
 '528, 574, 0.jpg',
 '288, 385, 0.jpg',
 '173, 440, 0.jpg',
 '384, 723, 0.jpg',
 '1497, 585, 0.jpg',
 '15, 607, 0.jpg',
 '543, 781, 0.jpg',
 '1188, 705, 0.jpg',
 '543, 555, 0.jpg',
 '810, 620, 0.jpg',
 '695, 455, 0.jpg',
 '159, 674, 0.jpg',
 '872, 783, 0.jpg',
 '777, 374, 0.jpg',
 '617, 453, 0.jpg',
 '191, 31, 0.jpg',
 '1552, 249, 0.jpg',
 '1520, 692, 0.jpg',
 '1271, 237, 0.jpg',
 '1791, 376, 0.jpg',
 '1769, 317, 0.jpg',
 '759, 501, 0.jpg',
 '529, 280, 0.jpg',
 '1376, 383, 0.jpg',
 '24, 208, 0.jpg',
 '1815, 806, 0.jpg',
 '1503, 367, 0.jpg',
 '1516, 12, 0.jpg',
 '711, 784, 0.jpg',
 '1759, 257, 0.jpg',
 '1900, 679, 0.jpg',
 '968, 184, 0.jpg',
 '482, 400, 0.jpg',
 '639, 183, 0.jpg',
 '

In [564]:
def index_to_path(row):
    index_filepath_found = '/Volumes/Extreme SSD/PROJECT MANACUS'
    possible_branches = row.index[:5]
    for child_folder_from_spreadsheet in possible_branches:
        index_filepath_found = os.path.join(str(index_filepath_found), str(row[child_folder_from_spreadsheet]))
    if os.path.exists(index_filepath_found):
        splits = index_filepath_found.split('/')
        lek, pista, = splits[5], splits[6]
        arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates') + f"/{lek}/{pista}/"
        if not os.path.exists(arena):
            os.makedirs(arena)
            #print('new directory made:', arena)
        return index_filepath_found
    else:
        return False
    

def filter_by_filepath_exists(spreadsheet):
    filtered_spreadsheet = spreadsheet[~pd.isna(spreadsheet['Folder'])]
    filtered_spreadsheet['filepath'] = filtered_spreadsheet.apply(index_to_path, axis=1)
    print(len(filtered_spreadsheet), 'videos found matching spreadsheet')
    return filtered_spreadsheet


def highlight(ssdfilepath):
    if os.path.exists(ssdfilepath):
        splits = ssdfilepath.split('/')
        lek, pista, vidname = splits[5], splits[6], splits[8]
        arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates') + f"/{lek}/{pista}/"
        if not os.path.exists(arena):
            os.makedirs(arena)

        all_centers = []
        
        #open_folder(arena)
        cap = cv2.VideoCapture(ssdfilepath)
        ret, prev_frame_color = cap.read()  # Initialize prev_frame with the first frame
        if not ret:
            print("Could not read the first frame.")
        frame_count = 0
        
        while cap.isOpened():
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 20)
            frame_count += 1
            ret, frame_color = cap.read()
            if not ret:
                break
            diff_color = cv2.absdiff(prev_frame_color, frame_color)
            diff_gray = cv2.cvtColor(diff_color, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(diff_gray, 30, 255, cv2.THRESH_BINARY)

            contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]             
                for box in [cv2.boundingRect(cnt) for cnt in contours]:
                    x, y, w, h = box
                    center = (x + w // 2, y + h // 2)

                    crop_x1 = max(center[0] - width, 0)
                    crop_x2 = min(center[0] + width, frame_color.shape[1])
                    crop_y1 = max(center[1] - width, 0)
                    crop_y2 = min(center[1] + width, frame_color.shape[0])

                    cropped_image = cv2.resize(frame_color[crop_y1:crop_y2, crop_x1:crop_x2], (100, 100))

                    
                    keep_dot = True
                    for c in all_centers:
                        distance = euclidean_distance(c, center)
                        if distance < 5:
                            keep_dot = False
                            break
                    
                    if keep_dot == True:
                        known_class = 0
                        all_centers.append(center)
                        image_filename = os.path.join(arena, f"unknown/{center[0]}, {center[1]}, {known_class}.jpg")
                        cv2.imwrite(image_filename, cropped_image)
                        cv2.circle(frame_color, center, 3, (0, 0, 255), -1)
                    cv2.imshow(f"{ssdfilepath.split('/')[5] + ' ' + ssdfilepath.split('/')[6]}", frame_color)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            prev_frame_color = frame_color  # Update prev_frame for the next iteration
        cap.release()
        cv2.destroyAllWindows()
        print(len(all_centers))
        
    else:
        print(f"Could not open video because ssdfilepath '{ssdfilepath}' does not exist.")
    

In [ ]:
import torch
import torch.nn as nn
class FirstNet(nn.Module):
    def __init__(self):
        super(FirstNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # Reduced filters here
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(16),  # BatchNorm after ReLU

            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # Reduced filters here
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(32),  # BatchNorm after ReLU

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Reduced filters here
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64),  # BatchNorm after ReLU
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64 * 12 * 12, 512),  # Reduced neurons here
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, 64),  # Reduced neurons here
            nn.ReLU(inplace=True),
            nn.Linear(64, 2),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

from torchvision import transforms
transform1 = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

class SecondNet(nn.Module):
    def __init__(self):
        super(SecondNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 25 * 25, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)
    
transform2 = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use CUDA if available
model1 = FirstNet()  
model1.load_state_dict(torch.load(os.path.expanduser('~/Desktop/ManacusManacusFiles/firstmodeliterations/0.8641975308641975_firstmodel.pth'), map_location=device))  
model1.to(device)
model1.eval()

# Load the second model
model2 = SecondNet()
model2.load_state_dict(torch.load(os.path.expanduser('~/Desktop/ManacusManacusFiles/secondmodel.pth'), map_location=device))
model2.to(device)
model2.eval()

print("\nModels loaded successfully")

In [565]:
spreadsheet = make_spreadsheet()
filtered = filter_by_filepath_exists(spreadsheet)
filtered_samp = sample_files(filtered, n=1)
for f in filtered['filepath']:
    highlight(f)

/var/folders/n3/j2btn17118v_1w899s6p1z480000gn/T/ipykernel_50786/1475050257.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_spreadsheet['filepath'] = filtered_spreadsheet.apply(index_to_path, axis=1)
/var/folders/n3/j2btn17118v_1w899s6p1z480000gn/T/ipykernel_50786/3747573441.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Place'] = temp['Lek'] + ' ' + temp['Pista']


8757 videos found matching spreadsheet
152
213
95
121
159
210
136
117
138
240
127
115
119
146
140


KeyboardInterrupt: 

: 

In [538]:

def save_image_by_desktop_dir(ssdfilepath):
    full_filenames = []
    if os.path.exists(ssdfilepath):
        splits = ssdfilepath.split('/')
        lek, pista, vidname = splits[5], splits[6], splits[8]
        arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/unknown/"
        if not os.path.exists(arena):
            os.makedirs(arena)

        all_centers = []
        cap = cv2.VideoCapture(ssdfilepath)
        ret, prev_frame_color = cap.read()  # Initialize prev_frame with the first frame
        if not ret:
            print("Could not read the first frame.")
        frame_count = 0
        
        while cap.isOpened():
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 20)
            frame_count += 1
            ret, frame_color = cap.read()
            if not ret:
                break
            diff_color = cv2.absdiff(prev_frame_color, frame_color)
            diff_gray = cv2.cvtColor(diff_color, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(diff_gray, 40, 255, cv2.THRESH_BINARY)

            contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]             
                for box in [cv2.boundingRect(cnt) for cnt in contours]:
                    x, y, w, h = box
                    center = (x + w // 2, y + h // 2)

                    crop_x1 = max(center[0] - width, 0)
                    crop_x2 = min(center[0] + width, frame_color.shape[1])
                    crop_y1 = max(center[1] - width, 0)
                    crop_y2 = min(center[1] + width, frame_color.shape[0])
                    
                    keep_dot = True
                    for c in all_centers:
                        distance = euclidean_distance(c, center)
                        if distance < 10:
                            keep_dot = False
                            break
                    
                    if keep_dot == True:
                        cropped_image = cv2.resize(frame_color[crop_y1:crop_y2, crop_x1:crop_x2], (100, 100))
                        diff_color_cropped = cv2.resize(diff_color[crop_y1:crop_y2, crop_x1:crop_x2], (100, 100))
                        all_centers.append(center)
                        
                        image_filename = os.path.join(arena, f"{center[0]}, {center[1]}.jpg")
                        #cv2.rectangle(frame_color, (all_centers[-1][0]-width, all_centers[-1][1]-width), (all_centers[-1][0]+width, all_centers[-1][1]+width), (0, 255, 0), 2)  # Green rectangle, thickness = 2
                        #cv2.imwrite(image_filename, cropped_image)
                        full_filenames.append(image_filename)
                        cv2.circle(frame_color, center, 10, (0, 0, 255), -1)
            
            cv2.imshow(f"{ssdfilepath.split('/')[5] + ' ' + ssdfilepath.split('/')[6]}", frame_color)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            prev_frame_color = frame_color  # Update prev_frame for the next iteration
        cap.release()
        cv2.destroyAllWindows()
        print(lek, pista, vidname, len(all_centers))
        return full_filenames
        
    else:
        print(f"Could not open video because ssdfilepath '{ssdfilepath}' does not exist.")

In [487]:
spreadsheet = make_spreadsheet(os.path.join(PROJECT_FOLDER, 'manacus_spreadsheets'))


In [483]:
spreadsheet = make_spreadsheet(os.path.join(PROJECT_FOLDER, 'manacus_spreadsheets'))
for indx in spreadsheet.index:
    ssdfilepath = index_to_path(spreadsheet, indx)
    if ssdfilepath != False:
        splits = ssdfilepath.split('/')
        cam, lek, pista, daterange, vidname = splits[4], splits[5], splits[6], splits[7], splits[8]
        arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"{lek}/{pista}"
        if not os.path.exists(arena):
            os.makedirs(arena)
    spreadsheet.loc[indx, 'filepath'] = ssdfilepath
print(len(spreadsheet), 'rows in spreadsheet')
spreadsheet = spreadsheet[spreadsheet['filepath'] != False]
print(len(spreadsheet), 'rows with working filepath')
spreadsheet = spreadsheet[(spreadsheet['FemVisitation'] == 1)].reset_index(drop=True)
sampled_spreadsheet = sample_files(spreadsheet, n=1)
print(len(sampled_spreadsheet), 'videos of interes')

10675 rows in spreadsheet
0 rows with working filepath
0 videos of interes


In [395]:
filenames = save_image_by_desktop_dir(filepath)
for file in filenames:
    img = cv2.imread(file)
    cv2.imshow(os.path.basename(file), img)
    
    action = None  # Initialize the action variable save_image_by_desktop_dir
    
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            action = 'quit'
            break
        elif key == ord('b'):
            action = '0'
            break
        elif key == ord('m'):
            action = '1'
            break
        elif key == ord('f'):
            action = '1'
            break
    
    cv2.destroyAllWindows()  # Close the image window
    
    if action == 'quit':
        break  # Exit the loop if 'q' is pressed
    elif action in ['0', '1', '2']:
        new_name = file[:file.find(os.path.basename(file))] + os.path.basename(file).split('.')[0] + f', {action}.jpg'
        print(new_name)
        cv2.imwrite(new_name, img)



KeyboardInterrupt: 

In [445]:
lek = 'Lek 14'
pista = 'Pista 1 (ORB)'
arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/"
os.listdir(arena)

SyntaxError: f-string: empty expression not allowed (4193601399.py, line 4)

In [481]:

for indx in sampled_spreadsheet.index:
    filepath = index_to_path(sampled_spreadsheet, indx)
    splits = ssdfilepath.split('/')
    lek, pista, vidname = splits[5], splits[6], splits[8]
    arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/"
    save_image_by_desktop_dir(filepath)
    

AttributeError: 'bool' object has no attribute 'split'

In [ ]:

for indx in sampled_spreadsheet.index:
    filepath = index_to_path(sampled_spreadsheet, indx)
    splits = ssdfilepath.split('/')
    lek, pista, vidname = splits[5], splits[6], splits[8]
    arena = os.path.expanduser(f'~/Desktop/TwoManakin/coordinates/') + f"/{lek}/{pista}/"
    filenames = save_image_by_desktop_dir(filepath)
    for file in filenames:
        img = cv2.imread(file)
        cv2.imshow(os.path.basename(file), img)
        
        action = None  # Initialize the action variable
        
        while True:
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                action = 'quit'
                break
            elif key == ord('b'):
                action = '0'
                break
            elif key == ord('m'):
                action = '1'
                break
            elif key == ord('f'):
                action = '2'
                break
        
        cv2.destroyAllWindows()  # Close the image window
        
        if action == 'quit':
            break  # Exit the loop if 'q' is pressed
        elif action in ['0', '1', '2']:
            new_name = file[:file.find(os.path.basename(file))] + os.path.basename(file).split('.')[0] + f', {action}.jpg'
            print(new_name)
            cv2.imwrite(new_name, img)
